In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.0 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata

dataset = pd.read_csv("/content/drive/MyDrive/movie_reviews.csv.bz2")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [5]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode('utf-8','ignore')
  return text

In [6]:
def pre_process_corpus(texts):
  preprocessed_text = []
  for doc in tqdm.tqdm(texts):
    doc = strip_html_tags(doc)
    #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
    doc = doc.lower()
    doc = remove_accented_characters(doc)
    #Do remove contractions as well and check the results
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc = re.sub(r' +',' ',doc)
    doc = doc.strip()
    #doc = "".join(doc)
    preprocessed_text.append(doc)
  return preprocessed_text

In [7]:
reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]


In [8]:
%%time
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 673/35000 [00:00<00:15, 2245.23it/s]<ipython-input-4-46650b581e77>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:08<00:00, 1807.14it/s]

CPU times: user 19.7 s, sys: 141 ms, total: 19.8 s
Wall time: 27.5 s


In [9]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',oov_token = 'UNK')

In [21]:
norm_train_reviews[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode youll be hooked they are right as this is exactly what happened with methe first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordit is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayi would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare forget pretty pictur

In [10]:
t.fit_on_texts(norm_train_reviews)

In [11]:
t.word_index['UNK'] = 0

In [12]:
train_sequences = t.texts_to_sequences(norm_train_reviews)
test_sequences = t.texts_to_sequences(norm_test_reviews)

In [ ]:
train_sequences[0]

In [15]:
len(t.word_index)

176890

In [16]:
t.document_count

35000

In [17]:
MAX_SEQUENCE_LENGTH = 1000

In [18]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences,maxlen = MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen = MAX_SEQUENCE_LENGTH)

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
num_classes = 2

In [23]:
y_train = le.fit_transform(train_sentiment)
y_test = le.transform(test_sentiment)

In [24]:
VOCAB_SIZE = len(t.word_index)

In [25]:
EMBEDDING_DIM = 300

In [29]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.GRU(128,return_sequences=False))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(X_train,y_train,epochs =2,batch_size = 100,shuffle = True,validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 45s 104ms/step - accuracy: 0.6664 - loss: 0.6024 - val_accuracy: 0.8407 - val_loss: 0.3947
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 37s 105ms/step - accuracy: 0.9226 - loss: 0.2026 - val_accuracy: 0.8815 - val_loss: 0.2848


In [33]:
score = model.evaluate(X_test,y_test)
print("Accuracy: ",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.8848 - loss: 0.2883
Accuracy:  88.15333247184753


In [34]:
embedding_layer = model.get_layer('embedding')

In [35]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word, idx in word_index.items():
  if idx <VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word, embedding in list(word_embeddings.items())[10:15]:
  print(f'Word: {word}, Embedding: {embedding}')

Word: i, Embedding: [-0.00539925 -0.01367678 -0.02983657 -0.03418371 -0.01613618  0.04503
  0.00827715  0.02524475 -0.0188555  -0.03432133 -0.0179191   0.0331763
 -0.01469888 -0.01173743  0.02749514 -0.00380506 -0.00804417  0.06400453
  0.01750441  0.03776209 -0.03621191 -0.01569487 -0.01854014 -0.02996394
  0.01991318 -0.01040881  0.04260404  0.02522049 -0.01786766 -0.03241784
 -0.00079969  0.00624748 -0.00656252  0.05214302 -0.0243632  -0.00530692
  0.07642546  0.00650386 -0.01468953  0.02107304 -0.03234603  0.033338
 -0.0321713   0.00608536  0.02215634  0.0056362  -0.03389269  0.03745358
 -0.02332874  0.04589349  0.01492599 -0.08124716  0.03414188 -0.0073094
  0.00873965  0.00687322  0.01176066  0.03051621  0.05194023 -0.04453512
  0.0247214   0.02554815  0.02629643 -0.02492854 -0.03383877  0.00918781
  0.0407359  -0.02549986 -0.05578196  0.07285712  0.04078901  0.03758783
  0.03484802  0.01640773 -0.0084839  -0.05433102  0.03277331  0.02143932
 -0.05888581  0.03057802 -0.04902388 -

In [38]:
word_embeddings['good']

array([ 6.62169140e-03, -3.82755846e-02, -8.95390008e-03,  1.70599688e-02,
       -4.23888414e-04,  6.97628558e-02,  4.40128669e-02,  1.53414859e-02,
       -1.73390824e-02, -3.20590846e-02, -7.47125745e-02,  4.77860793e-02,
       -5.25690727e-02, -5.01233377e-02,  2.67270743e-03,  2.02439222e-02,
        2.51235086e-02,  2.08378341e-02,  9.03537124e-03,  2.13452708e-02,
       -6.54197484e-03, -3.45219229e-03,  4.80022281e-05, -2.58062277e-02,
        3.59780602e-02,  1.49994530e-02, -2.12820489e-02, -7.63605610e-02,
       -2.24578027e-02,  2.21043229e-02,  6.63881153e-02,  2.47939490e-03,
       -4.59611835e-03,  2.09681857e-02, -4.36918028e-02,  6.11550175e-03,
        2.01940406e-02, -7.58442003e-03, -2.39894744e-02, -9.48134810e-02,
        2.52138749e-02,  5.48775727e-03, -4.07170281e-02,  4.92653698e-02,
       -3.98882888e-02,  3.01728975e-02,  2.07668301e-02, -9.20865871e-03,
       -1.50745427e-02,  4.84044757e-03, -1.90226361e-02,  2.58528329e-02,
        4.25519757e-02, -

In [39]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [40]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(X_train,y_train,epochs =2,batch_size = 100,shuffle = True,validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 78s 210ms/step - accuracy: 0.7327 - loss: 0.4999 - val_accuracy: 0.8641 - val_loss: 0.3379
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 81s 209ms/step - accuracy: 0.9230 - loss: 0.2133 - val_accuracy: 0.8727 - val_loss: 0.3051


In [42]:
score = model.evaluate(X_test,y_test)
print("Accuracy: ",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 31ms/step - accuracy: 0.8729 - loss: 0.3079
Accuracy:  87.26666569709778


In [43]:
embedding_layer = model.get_layer('embedding_1')

In [44]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word, idx in word_index.items():
  if idx <VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word, embedding in list(word_embeddings.items())[10:15]:
  print(f'Word: {word}, Embedding: {embedding}')

Word: i, Embedding: [ 0.02161017  0.01016464 -0.0344336   0.00122708  0.00485192  0.02467518
  0.04477451  0.00276823 -0.03736994  0.01093021  0.00318434  0.01560665
  0.04214952  0.02867384  0.016883   -0.04266184  0.06286643  0.05429688
  0.04263742  0.0273772  -0.0190854  -0.04935349  0.05905822  0.04074043
 -0.03213617  0.00608135  0.02824392  0.00992134 -0.02297647  0.03575327
  0.01406809 -0.01752631  0.02756415  0.03398655  0.01430477 -0.02097004
 -0.0233241   0.0599719   0.00332613  0.03170693  0.04380341  0.01014346
 -0.0273118   0.00129605  0.02994707 -0.03650814 -0.02554997 -0.03154456
 -0.06282184  0.04919599  0.00662109  0.02789403  0.04158591 -0.05145029
 -0.00323008  0.01909375 -0.01300988 -0.00609357  0.0212041  -0.02955449
  0.03869161 -0.02134361 -0.050593   -0.02602028 -0.05548182  0.02450823
 -0.03569289  0.00362536 -0.01686659  0.04656775 -0.01811399  0.07473809
  0.04502587  0.00261415 -0.01302208 -0.04809745 -0.02153728 -0.02144432
 -0.01196803 -0.01013983  0.022

In [45]:
word_embeddings['good']

array([ 0.06350393,  0.00067502,  0.02192595,  0.01426579,  0.03351908,
        0.05234223,  0.00104904,  0.02680966,  0.03030318, -0.04553887,
        0.00551062,  0.02396228,  0.0305665 , -0.01814475,  0.00964894,
        0.03333849, -0.00019453,  0.00850854, -0.00123757,  0.01457866,
       -0.04432176, -0.01656258,  0.03194575, -0.00561726, -0.05521175,
        0.02958415, -0.00816367, -0.05078987, -0.00079042,  0.06101362,
       -0.03150417,  0.02762431,  0.01701283,  0.06158182, -0.05484222,
       -0.06425846, -0.055809  ,  0.01942469, -0.02189567,  0.02848111,
        0.00120583, -0.00581923, -0.00374039, -0.03899815,  0.03400095,
        0.04431776, -0.04683483,  0.0411395 , -0.05120998,  0.02064977,
       -0.00542242,  0.01063838, -0.04299626, -0.07393159,  0.00366555,
       -0.03824726,  0.02371001,  0.01854894, -0.02519264,  0.01534188,
        0.06040534, -0.08149768,  0.05007304, -0.05455255, -0.03003257,
       -0.06575117,  0.01197545,  0.02827532,  0.01569667, -0.06

In [47]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

In [48]:
model.fit(X_train,y_train,epochs =2,batch_size = 100,shuffle = True,validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 232s 647ms/step - accuracy: 0.7457 - loss: 0.4938 - val_accuracy: 0.8273 - val_loss: 0.4309
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 226s 647ms/step - accuracy: 0.9060 - loss: 0.2487 - val_accuracy: 0.7145 - val_loss: 0.5689
